In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd workspace/theislab/mubind/notebooks

/mnt/c/Users/ignacio.ibarra/Dropbox/workspace/theislab/mubind/notebooks


In [3]:
import mubind as mb
import numpy as np
import pandas as pd
import torch
import bindome as bd
bd.constants.ANNOTATIONS_DIRECTORY = 'annotations'
# mb.models.MultiBind
import torch.optim as topti
import torch.utils.data as tdata
import matplotlib.pyplot as plt
import logomaker
import os
import scipy
import pickle

# Use a GPU if available, as it should be faster.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

/home/rio/miniconda3/envs/mubind/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda:0


In [4]:
adata = mb.bindome.datasets.scATAC.PBMCs_10x_v2(datadir='../../atac_poisson_study/data/')
peak_ids = adata.var_names

In [5]:
adata.shape

(10246, 165434)

In [6]:
extend = 50

In [7]:
adata.var['summit'] = ((adata.var['end'] + adata.var['start']) / 2).astype(int)
adata.var['summit.start'] = adata.var['summit'] - extend
adata.var['summit.end'] = adata.var['summit'] + extend

In [8]:
adata.var['k.summit'] = adata.var['chr'] + ':' + adata.var['summit.start'].astype(str) + '-' + adata.var['summit.end'].astype(str)

In [9]:
adata.shape

(10246, 165434)

In [10]:
n_seqs = 25000 # adata.shape[1] # None # 1000
seqs = mb.bindome.tl.get_sequences_from_bed(adata.var[['chr', 'summit.start', 'summit.end']].head(n_seqs), genome='hg38', uppercase=True)
keys = set([s[0] for s in seqs])
adata = adata[:,adata.var['k.summit'].isin(keys)]
# seqs = [[s[0], s[1].upper()] for s in seqs[0]]

/tmp/tmp1f74wcw1
genome hg38 False
options
/mnt/c/Users/ignacio.ibarra/Dropbox/annotations/hg38/genome/hg38.fa
True /mnt/c/Users/ignacio.ibarra/Dropbox/annotations/hg38/genome/hg38.fa
running bedtools...
bedtools getfasta -fi /mnt/c/Users/ignacio.ibarra/Dropbox/annotations/hg38/genome/hg38.fa -bed /tmp/tmp1f74wcw1 -fo /tmp/tmpgvv0x_n8


In [11]:
len(seqs)

25000

In [12]:
# remove Ns
for s in seqs:
    if 'N' in s:
        assert False
    # seqs = [[s[0], s[1].replace('N', '')] for s in seqs]
counts = adata.X.T

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 1., 2., ..., 2., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [38]:
next_data = pd.DataFrame(counts.A) # sparse.from_spmatrix(counts.A)
# next_data = pd.DataFrame.sparse.from_spmatrix(counts)# var = []
# for ri, r in next_data.iterrows():
#     if ri % 1000 == 0:
#         print(ri, next_data.shape)
#     # print(ri, r.values[:-1], r.values[:-1].var())
#     var.append(r.values[:-1].var())
#     # break



In [39]:
next_data['var'] = next_data.var(axis=1)
# next_data = next_data[range(100)].copy()

In [40]:
next_data.index = [s[1] for s in seqs]
next_data.index.name = 'seq'

In [41]:
# sum_index = next_data[next_data.columns[:-1]].var(axis=1).sort_values(ascending=False).index

In [42]:
n_cells = 15000
top_var = next_data[['var']].sort_values('var', ascending=False).index[:n_cells]

In [43]:
# next_data = next_data.head(10000)
next_data_sel = next_data.reindex(top_var) # .reset_index(drop=True)

In [44]:
next_data_sel

,0,1,2,3,4,5,6,7,8,9,...,10237,10238,10239,10240,10241,10242,10243,10244,10245,var
seq,,,,,,,,,,,,,,,,,,,,,
CGCCCCAAGGATTCGCTCTTGCGTCACCGCCTCGCCTTCAGGACGCCTCCGCGCCTCGCCCTGTCCCCAGGGTCAATTCTGGGGTCACCGCCTGTCCCCA,3.0,1.0,6.0,5.0,4.0,1.0,9.0,1.0,3.0,5.0,...,4.0,1.0,3.0,0.0,6.0,1.0,2.0,2.0,1.0,3.954389
ACTGACTGTCTCCGTCTTCGCCCCTGCCCTGGCCCCGCCCCCTGCCTGCTCTGGCCCCGCCCCGCCGCTCTCTCCGTCTCCGCCCCTGCCCTGGCCCCAC,3.0,4.0,3.0,4.0,2.0,1.0,5.0,3.0,3.0,3.0,...,2.0,0.0,3.0,7.0,2.0,3.0,2.0,2.0,0.0,3.728141
AAGGCAAACTACAACTCCCAAAGGCCCTTGCGCGGACTCAGCGCGCGAGGGACGAAGGGCGGTCCACTGTAAGGGGAAATGCTGAGGATCACTGGGCTCC,1.0,0.0,3.0,2.0,1.0,4.0,5.0,1.0,1.0,1.0,...,3.0,1.0,3.0,3.0,4.0,4.0,5.0,0.0,2.0,3.250077
GGCGACCGCGGCGCTGCTTCGAGGCGCCACTCCGGGGCGCGGCGGCCCGGTCTGGCGCTGGCGGCTGCGCGCGGCCCCAAGGTGCCGCCTGGCCCACAGC,2.0,2.0,2.0,4.0,2.0,3.0,4.0,1.0,2.0,2.0,...,0.0,1.0,6.0,1.0,3.0,3.0,1.0,2.0,1.0,3.166806
ACCCTCCCCCGGGGTGGTGCCGGAGGGGGGGATGATGGGTCGAGGGGTGTGTTTATGTGGAAGCGAGATGACCGGCAGGAACCTGCCCCAATGGGCTGCA,3.0,1.0,3.0,5.0,1.0,4.0,8.0,1.0,3.0,1.0,...,2.0,0.0,0.0,3.0,2.0,4.0,3.0,3.0,2.0,3.141236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AGGCATCAGGGATACAGTGGTGAACAAGCATACACAATCTCTGCCCTCATGGAGCTTAGATTCTTGGCCAACTCATCCCAGCATGGTGTCTCCTCCTCCA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.022699
AAGGGCCATTGTACTTGCTATTTCCTTTGCCTCTAAGTGCACCTCCTCTAACATTTTGCATGAGTCATTCCTTCTCATTATTTAGGTCTTAGCTCAAATG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022699
CCTGTATTTTTAAATACACACGTGGGAAAAATTATAAACAAAGGGAGGAAAATCCTTTAGGAAAAAATGACTTTTGCCAAGCAACTATTTAGTCTCTGCT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022693


In [45]:
del next_data_sel['var']
# next_data_sel.index = next_data_sel['seq']
# del next_data_sel['seq']

In [46]:
next_data_sel

,0,1,2,3,4,5,6,7,8,9,...,10236,10237,10238,10239,10240,10241,10242,10243,10244,10245
seq,,,,,,,,,,,,,,,,,,,,,
CGCCCCAAGGATTCGCTCTTGCGTCACCGCCTCGCCTTCAGGACGCCTCCGCGCCTCGCCCTGTCCCCAGGGTCAATTCTGGGGTCACCGCCTGTCCCCA,3.0,1.0,6.0,5.0,4.0,1.0,9.0,1.0,3.0,5.0,...,3.0,4.0,1.0,3.0,0.0,6.0,1.0,2.0,2.0,1.0
ACTGACTGTCTCCGTCTTCGCCCCTGCCCTGGCCCCGCCCCCTGCCTGCTCTGGCCCCGCCCCGCCGCTCTCTCCGTCTCCGCCCCTGCCCTGGCCCCAC,3.0,4.0,3.0,4.0,2.0,1.0,5.0,3.0,3.0,3.0,...,0.0,2.0,0.0,3.0,7.0,2.0,3.0,2.0,2.0,0.0
AAGGCAAACTACAACTCCCAAAGGCCCTTGCGCGGACTCAGCGCGCGAGGGACGAAGGGCGGTCCACTGTAAGGGGAAATGCTGAGGATCACTGGGCTCC,1.0,0.0,3.0,2.0,1.0,4.0,5.0,1.0,1.0,1.0,...,2.0,3.0,1.0,3.0,3.0,4.0,4.0,5.0,0.0,2.0
GGCGACCGCGGCGCTGCTTCGAGGCGCCACTCCGGGGCGCGGCGGCCCGGTCTGGCGCTGGCGGCTGCGCGCGGCCCCAAGGTGCCGCCTGGCCCACAGC,2.0,2.0,2.0,4.0,2.0,3.0,4.0,1.0,2.0,2.0,...,1.0,0.0,1.0,6.0,1.0,3.0,3.0,1.0,2.0,1.0
ACCCTCCCCCGGGGTGGTGCCGGAGGGGGGGATGATGGGTCGAGGGGTGTGTTTATGTGGAAGCGAGATGACCGGCAGGAACCTGCCCCAATGGGCTGCA,3.0,1.0,3.0,5.0,1.0,4.0,8.0,1.0,3.0,1.0,...,1.0,2.0,0.0,0.0,3.0,2.0,4.0,3.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AGGCATCAGGGATACAGTGGTGAACAAGCATACACAATCTCTGCCCTCATGGAGCTTAGATTCTTGGCCAACTCATCCCAGCATGGTGTCTCCTCCTCCA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
AAGGGCCATTGTACTTGCTATTTCCTTTGCCTCTAAGTGCACCTCCTCTAACATTTTGCATGAGTCATTCCTTCTCATTATTTAGGTCTTAGCTCAAATG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CCTGTATTTTTAAATACACACGTGGGAAAAATTATAAACAAAGGGAGGAAAATCCTTTAGGAAAAAATGACTTTTGCCAAGCAACTATTTAGTCTCTGCT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
df = next_data_sel.copy()
df = df[df.sum(axis=1) != 0]
# df2 = []
# for i in df.columns:
#     df3 = df[[i]].copy()
#     df3 = df3.head(100)
#     df3.columns = [0]
#     df3[1] = 1 # need a minimal non-zero count
#     df3['batch'] = i
#     df2.append(df3)
# df2 = pd.concat(df2)
# # df2['is_count_data'] = True
# dataset = mb.datasets.SelexDataset(df2, n_rounds=2, labels=[0, 1], enr_series=False)
dataset = mb.datasets.SelexDataset(df, n_rounds=df.shape[1], enr_series=False)


In [48]:
df.shape

(15000, 10246)

In [49]:
# df = next_data_sel.copy()
# dataset = mb.datasets.GenomicsDataset(df)
# train = tdata.DataLoader(dataset=dataset, batch_size=1000, shuffle=True)

train = tdata.DataLoader(dataset=dataset, batch_size=512, shuffle=True)

In [ ]:
model, best_loss = mb.tl.optimize_iterative(train, device, num_epochs=500, show_logo=False, use_mono=True, use_dinuc=False, opt_kernel_shift=0,
                                                 early_stopping=50, log_each=10, n_kernels=3, w=20,) #  target_dim=train.dataset.signal.shape[1])

next w 20 <class 'int'>
# rounds {10246}
# rounds {10246}
# use_mono True
# use_dinuc False
# dinuc_mode local
# batches 1
# kernels 3
# initial w 20
# enr_series False

Kernel to optimize 0

FREEZING KERNELS
optimizing feature type mono
setting grad status of kernel (mono, dinuc) at 0 to (1, 0)
setting grad status of kernel (mono, dinuc) at 1 to (0, 0)
setting grad status of kernel (mono, dinuc) at 2 to (0, 0)


kernels mask None
optimizer:  Adam 
criterion: PoissonLoss 
# epochs: 500 
early_stopping: 50
lr= 0.01, weight_decay= 0.001, dir weight= 0


In [ ]:
r2 = mb.tl.scores(model, train, by='batch')
print("R^2:", r2)


In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 20, 5
rcParams['figure.dpi'] = 100
mb.pl.conv(model, n_cols=2, show=True)
plt.show()
